In [ ]:
using Pkg
pkg"activate ."

In [ ]:
# Run this cell in order to download all the package dependencies with the exact versions used in the book
# This is necessary if (some of) the packages have been updated and have introduced breaking changes
pkg"instantiate"

In [ ]:
using CSV, DataFrames

In [ ]:
df = CSV.read("Map_of_Registered_Business_Locations.csv")

In [ ]:
describe(df)

In [ ]:
size(df, 1)

In [ ]:
df[df[Symbol("Parking Tax")] .== true, :][1:10, [Symbol("DBA Name"), Symbol("Parking Tax")]]

In [ ]:
using Pkg
pkg"add Query"

In [ ]:
using Query

In [ ]:
@from i in df begin
	@where i[Symbol("Parking Tax")] == true
	@select i
	@collect DataFrame
end

In [ ]:
rename!(df, [n => replace(string(n), " "=>"_") |> Symbol for n in names(df)])

In [ ]:
@from i in df begin
	@where i.Parking_Tax == true
	@select i
	@collect DataFrame
end

In [ ]:
assign = :(x = 2)

In [ ]:
eval(assign)

In [ ]:
x

In [ ]:
fieldnames(typeof(assign))

In [ ]:
dump(assign)

In [ ]:
assign.args[2] = 3

In [ ]:
eval(assign)

In [ ]:
x

In [ ]:
assign4 = Expr(:(=), :x, 4)

In [ ]:
eval(assign4)

In [ ]:
x

In [ ]:
quote
	y = 42
	x = 10
end

In [ ]:
eval(ans)

In [ ]:
y

In [ ]:
x

In [ ]:
name = "Dan"

In [ ]:
greet = :("Hello " * $name)

In [ ]:
eval(greet)

In [ ]:
macro greet(name)
    :("Hello " * $name)
end

In [ ]:
@greet("Adrian")

In [ ]:
@greet "Julia"

In [ ]:
macro twostep(arg)
	println("I execute at parse time. The argument is: ", arg)
	return :(println("I execute at runtime. The argument is: ", $arg))
end

In [ ]:
ex = macroexpand(@__MODULE__, :(@twostep :(1, 2, 3)));

In [ ]:
eval(ex)

In [ ]:
shopping_list = DataFrame(produce=["Apples", "Milk", "Bread"], qty=[5, 2, 1])

In [ ]:
@from p in shopping_list begin
    @select p
end

In [ ]:
@from p in shopping_list begin
    @select p.produce
end

In [ ]:
@from p in shopping_list begin
	@select p.produce
end

In [ ]:
@from p in shopping_list begin
	@select uppercase(p.produce), 2p.qty
end

In [ ]:
@from p in shopping_list begin
    @select { produce = uppercase(p.produce), qty = 2p.qty }
end

In [ ]:
@from p in shopping_list begin
    @select { PRODUCE = uppercase(p.produce), double_qty = 2p.qty }
    @collect
end

In [ ]:
@from p in shopping_list begin
    @select {PRODUCE = uppercase(p.produce), double_qty = 2p.qty}
    @collect DataFrame
end

In [ ]:
@from p in shopping_list begin
    @where p.qty < 2
    @select p
    @collect DataFrame
end

In [ ]:
@from p in shopping_list begin
    @let weekly_qty = 7p.qty
    @where weekly_qty > 10
    @select { p.produce, week_qty=weekly_qty }
    @collect DataFrame
end

In [ ]:
products_info = DataFrame(produce = ["Apples", "Milk", "Bread"], price = [2.20, 0.45, 0.79], allergenic = [false, true, true])

In [ ]:
shopping_info = @from p in shopping_list begin
	@join pinfo in products_info on p.produce equals pinfo.produce
	@select { p.produce, p.qty, pinfo.price, pinfo.allergenic }
	@collect DataFrame
end

In [ ]:
@from p in shopping_info begin
    @group p.produce by p.allergenic
    @collect
end

In [ ]:
@from p in shopping_info begin
   @group p by p.allergenic into q
   @select { allergenic = key(q), count = length(q.allergenic), produce = join(q.produce, ", ") }
   @collect DataFrame
end

In [ ]:
@from p in products_info begin
    @orderby descending(p.price), p.produce
    @select p
    @collect DataFrame
end

In [ ]:
pkg"add DataValues"
using DataValues

In [ ]:
clean_df = @from b in df begin
        @where lowercase(b.City) == "san francisco" && b.State == "CA" && 
            ! isna(b.Street_Address) && ! isna(b.Source_Zipcode) && 
            ! isna(b.NAICS_Code) && ! isna(b.NAICS_Code_Description) && 
            ! isna(b.Business_Location) && 
            occursin(r"\((.*), (.*)\)", get(b.Business_Location)) && 
            isna(b.Business_End_Date) && isna(b.Location_End_Date)
    @select { b.DBA_Name, b.Source_Zipcode, b.NAICS_Code, b.NAICS_Code_Description, b.Business_Location }
    @collect DataFrame
end

In [ ]:
clean_df_geo = @from b in clean_df begin
    @let geo  = split(match(r"(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)", get(b.Business_Location)).match, ", ")
    @select {b.DBA_Name, b.Source_Zipcode, b.NAICS_Code, b.NAICS_Code_Description, lat = parse(Float64, geo[1]), long = parse(Float64, geo[2])}
    @collect DataFrame
end

In [ ]:
describe(clean_df_geo)

In [ ]:
unique(clean_df_geo[:, :Source_Zipcode]) |> length

In [ ]:
using Pkg
pkg"add Clustering"
using Clustering

In [ ]:
using CSV, DataFrames, Query
clean_df_geo = CSV.read("clean_df_geo.tsv", delim = '\t', nullable = false)

In [ ]:
model_data = @from b in clean_df_geo begin
	@group b by b.Source_Zipcode into g
	@let bcount = Float64(length(g))
	@orderby descending(bcount)
	@select { zipcode = Float64(get(key(g))), businesses_count = bcount }
	@collect DataFrame
end

In [ ]:
tail(model_data)

In [ ]:
using Gadfly

In [ ]:
plot(model_data, x=:businesses_count, Geom.histogram)

In [ ]:
model_data = @from b in clean_df_geo begin
	@group b by b.Source_Zipcode into g
	@let bcount = Float64(length(g))
    @where bcount > 10
	@orderby descending(bcount)
	@select { zipcode = Float64(get(key(g))), businesses_count = bcount }
	@collect DataFrame
end

In [ ]:
training_data = permutedims(convert(Array, model_data), [2, 1])

In [ ]:
result = kmeans(training_data, 3, init=:kmpp, display=:iter)

In [ ]:
result.assignments

In [ ]:
model_data[:cluster_id] = result.assignments

In [ ]:
model_data

In [ ]:
plot(model_data, y = :zipcode, x = :businesses_count, color = result.assignments, Geom.point, Scale.x_continuous(minvalue=0, maxvalue=5000), Scale.y_continuous(minvalue=94050, maxvalue=94200), Scale.x_continuous(format=:plain))

In [ ]:
companies_in_top_areas = @from c in clean_df_geo begin
    @where in(c.Source_Zipcode, [94110, 94103, 94109])
    @select c
    @collect DataFrame
end

In [ ]:
plot(companies_in_top_areas, y = :long, x = :lat, Geom.point, Scale.x_continuous(minvalue=36, maxvalue=40), Scale.y_continuous(minvalue=-125, maxvalue=-120), color=:Source_Zipcode)

In [ ]:
companies_in_top_areas = @from c in companies_in_top_areas begin
	@where c.lat != minimum(companies_in_top_areas[:lat])
	@select c
	@collect DataFrame
end

In [ ]:
plot(companies_in_top_areas, y = :long, x = :lat, Geom.point, Scale.x_continuous(minvalue=36, maxvalue=40), Scale.y_continuous(minvalue=-125, maxvalue=-120), color=:Source_Zipcode)

In [ ]:
activities = @from c in companies_in_top_areas begin
	@group c by c.NAICS_Code_Description into g
	@orderby descending(length(g))
	@select { activity = key(g), number_of_companies = length(g) }
	@collect DataFrame
end

In [ ]:
plot(activities, y=:number_of_companies, Geom.bar, color=:activity, Scale.y_continuous(format=:plain), Guide.XLabel("Activities"), Guide.YLabel("Number of companies"))

In [ ]:
model_data = @from c in companies_in_top_areas begin
	@select { latitude = c.lat, longitude = c.long }
	@collect DataFrame
end

In [ ]:
training_data = permutedims(convert(Array{Float64}, model_data), [2, 1])

In [ ]:
result = kmeans(training_data, 12, init=:kmpp, display=:iter)

In [ ]:
result.counts

In [ ]:
plot(result.counts, Geom.bar, y=result.counts, Guide.YLabel("Number of businesses"), Guide.XLabel("Cluster ID"), color=result.counts)

In [ ]:
companies_in_top_areas[:cluster_id] = result.assignments

In [ ]:
plot(companies_in_top_areas, color=:cluster_id, x=:lat, y=:long)

In [ ]:
export_data = @from c in companies_in_top_areas begin
                  @select { Name = c.DBA_Name, Zip = c.Source_Zipcode,
                              Group = string("Cluster $(c.cluster_id)"),
                                Latitude = c.lat, Longitude = c.long, City = "San Francisco",
                                   State = "CA" }
                @collect DataFrame
end

In [ ]:
CSV.write("businesses.csv", head(export_data, 250))